In [224]:
def data_generator(info):
    a = np.sin(np.array(info)) - np.cos(np.array(info))
    a[4] = 1
    return a

def sensitivity(func, delta_variable, variable_to_delta = [-1]):
    dy = np.empty(len(delta_variable))
    delta_variable = np.array(delta_variable)
    sim_range = np.linspace(0.25, 1.75, 8)
    dx = sim_range[1] - sim_range[0]
    
    if max(variable_to_delta) > 0:
        delta_matrix = np.ones(len(delta_variable))
        for i in len(sim_range):
            delta_matrix[variable_to_delta] *= sim_range
            sim_range[i] = delta_matrix 
        print(sim_range)
        
    y = func(delta_variable * (sim_range[0]))
    
    for i in range(len(sim_range)-1):
        new_y = func(delta_variable * (sim_range[i+1]))
        y = np.c_[y, new_y]
        
        if i > 0:
            #dy/dx
            a = (y[:,i] - y[:, i+1])
            b = (delta_variable[:] * sim_range[i] - delta_variable[:] * sim_range[i+1])
            b = np.transpose(b)
            
            dy_var = (a / b)
            print(np.shape(dy_var[0,:]))
            dy = np.c_[dy, dy_var[0,:]]
            
    for p in range(len(delta_variable)):
        plt.plot(sim_range[1:], dy[p,:], 'o', label = p,)
    plt.legend()
    print(dy[25,:], y[25])
    
    return sim_range, dy

In [225]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

info = pd.read_csv("digester_initial.csv")
info = info.transpose()
sensitivity(data_generator, info, [4])
a=1

ValueError: non-broadcastable output operand with shape (1,) doesn't match the broadcast shape (8,)

In [220]:
a = np.array([1,2,3,4])
b = a/2

c = [0, 3]

if max(c) > 0:
    a[c]*=2
    print(a)

[2 2 3 8]
